# Installation

In [14]:
# !conda clean --all
# !conda update -n base -c defaults conda

# CUDA Toolkit 11.3
# https://developer.nvidia.com/cuda-11.3.0-download-archive?target_os=Windows&target_arch=x86_64&target_version=10&target_type=exe_local
#https://pytorch.org/get-started/previous-versions/

# !conda create --name vulscriber-cluster-env python=3.8
# !conda install pytorch==1.10.0 torchvision==0.11.1 torchaudio==0.10.0 cudatoolkit=11.3 -c pytorch
# !conda install transformers pandas scikit-learn

# https://stackoverflow.com/questions/75242037/failed-to-import-transformers-onnx-config
# !conda install -c conda-forge sentence-transformers
# !conda install Pillow

# Load VULNERABLE Dataset

In [3]:
import json
import pandas as pd

# Đường dẫn file JSON gốc (cập nhật lại đường dẫn theo hệ thống của bạn)
input_file = r"C:\Users\Administrator\Downloads\mapr\VulScribeR-main\dataset\megavul.json"

# Load file JSON và chuyển thành DataFrame
with open(input_file, "r", encoding="utf-8") as f:
    json_data = json.load(f)
mega_vul_df = pd.DataFrame(json_data)



: 

In [ ]:

# Tách dữ liệu dựa trên trường 'is_vul'
vul_df = mega_vul_df[mega_vul_df['is_vul'] == True]
clean_df = mega_vul_df[mega_vul_df['is_vul'] == False]

# Đường dẫn cho file output (cập nhật lại đường dẫn nếu cần)
# vul_output_file = r"C:\Users\Administrator\Downloads\mapr\VulScribeR-main\dataset\megavul_vul.json"
# clean_output_file = r"C:\Users\Administrator\Downloads\mapr\VulScribeR-main\dataset\megavul_clean.json"

# # Ghi kết quả ra file JSON với định dạng 'records'
# vul_df.to_json(vul_output_file, orient="records", force_ascii=False, lines=True)
# clean_df.to_json(clean_output_file, orient="records", force_ascii=False, lines=True)

# print("Tách file thành công!")

MemoryError: 

In [1]:
import json
import pandas as pd

# Đường dẫn file JSON gốc (cập nhật lại đường dẫn theo hệ thống của bạn)
input_file = r"C:\Users\Administrator\Downloads\mapr\VulScribeR-main\dataset\simple\megavul_vul.json"

# Load file JSON và chuyển thành DataFrame
with open(input_file, "r", encoding="utf-8") as f:
    json_data = json.load(f)
vul_df = pd.DataFrame(json_data)


# Tokenizer

In [2]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")
model.to(device)


c:\ProgramData\miniconda3\envs\vulscriber-cluster-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\ProgramData\miniconda3\envs\vulscriber-cluster-env\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [ ]:
# # Encode vectors
# from tqdm import tqdm
# import torch
# import numpy as np
# import pickle
# import gc

# BATCH_SIZE = 128  
# SAVE_PATH = "encoded_vectors.pkl"  # Dùng pickle thay vì JSON
# SAVE_EVERY = 50  

# torch.backends.cudnn.benchmark = True  

# # Tải checkpoint nếu có
# try:
#     with open(SAVE_PATH, "rb") as f:
#         index_to_vector_map = pickle.load(f)
#         print(f"✅ Resume từ checkpoint: {len(index_to_vector_map)} vectors")
# except FileNotFoundError:
#     index_to_vector_map = {}

# # Chuẩn bị danh sách các mẫu chưa encode trước vòng lặp
# code_samples = [(i, str(row["func"])) for i, row in vul_df.iterrows() if str(row["func"]).strip()]
# code_samples = [(i, code) for i, code in code_samples if i not in index_to_vector_map]  
# total_samples = len(code_samples)

# # Nếu tất cả đã encode, thoát sớm
# if total_samples == 0:
#     print("✅ Tất cả mẫu đã được encode!")
#     exit()

# batch_count = 0  

# for i in tqdm(range(0, total_samples, BATCH_SIZE), desc="Encoding..."):
#     batch_indices, batch_codes = zip(*code_samples[i : i + BATCH_SIZE])

#     # Encode batch
#     inputs = tokenizer(batch_codes, return_tensors="pt", truncation=True, padding=True, max_length=512)
#     inputs = {k: v.to("cuda:0", non_blocking=True) for k, v in inputs.items()}

#     with torch.inference_mode(), torch.cuda.amp.autocast():
#         outputs = model(**inputs)

#     batch_vectors = outputs.last_hidden_state[:, 0, :].cpu().numpy()

#     # Lưu vào dict
#     index_to_vector_map.update({idx: vec.tolist() for idx, vec in zip(batch_indices, batch_vectors)})

#     batch_count += 1
#     if batch_count % SAVE_EVERY == 0:  # Chỉ lưu mỗi 10 batch
#         with open(SAVE_PATH, "wb") as f:
#             pickle.dump(index_to_vector_map, f)

#         torch.cuda.empty_cache()  
#         gc.collect()

# # Lưu lần cuối
# with open(SAVE_PATH, "wb") as f:
#     pickle.dump(index_to_vector_map, f)

# print(f"✅ Hoàn tất encoding! Đã lưu vào {SAVE_PATH}")


c:\ProgramData\miniconda3\envs\vulscriber-cluster-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [5]:
# # [DEBUG]
# from tqdm import tqdm
# import torch
# import numpy as np
# import pickle
# import gc

# BATCH_SIZE = 128  
# SAVE_PATH = "encoded_vectors.pkl"  # Dùng pickle thay vì JSON
# SAVE_EVERY = 50  

# torch.backends.cudnn.benchmark = True  

# # Tải checkpoint nếu có
# try:
#     with open(SAVE_PATH, "rb") as f:
#         index_to_vector_map = pickle.load(f)
#         print(f"✅ Resume từ checkpoint: {len(index_to_vector_map)} vectors")
# except FileNotFoundError:
#     index_to_vector_map = {}

# # Nếu muốn encode cả vulnerable và clean, bạn có thể quyết định chọn trường dựa trên 'is_vul'
# # Ví dụ: dùng func_before cho vulnerable, func cho clean
# def get_code(row):
#     if row.get("is_vul", False):
#         # Nếu có func_before, ưu tiên dùng nó
#         code = str(row.get("func_before", "")).strip()
#         if code:
#             return code
#     # Ngược lại, dùng func (có thể là clean hoặc đã fixed)
#     return str(row.get("func", "")).strip()

# # Kiểm tra vài mẫu để debug
# sample_debug = vul_df.head(5)
# print("Ví dụ mẫu ban đầu:")
# for idx, row in sample_debug.iterrows():
#     print(f"Index {idx} - is_vul: {row.get('is_vul')} - func: {row.get('func')[:50]}... - func_before: {row.get('func_before', '')[:50]}...")

# # Tạo danh sách mẫu để encode
# code_samples = [(i, get_code(row)) for i, row in vul_df.iterrows()]
# # Loại bỏ những mẫu rỗng
# code_samples = [(i, code) for i, code in code_samples if code]
# print(f"Tổng số mẫu sau khi lọc: {len(code_samples)}")

# # Loại bỏ các mẫu đã được encode từ checkpoint
# code_samples = [(i, code) for i, code in code_samples if i not in index_to_vector_map]
# total_samples = len(code_samples)
# print(f"Số mẫu cần encode sau khi loại bỏ checkpoint: {total_samples}")

# # Nếu tất cả đã encode, thoát sớm
# if total_samples == 0:
#     print("✅ Tất cả mẫu đã được encode!")
#     exit()

# batch_count = 0  

# for i in tqdm(range(0, total_samples, BATCH_SIZE), desc="Encoding..."):
#     batch_indices, batch_codes = zip(*code_samples[i : i + BATCH_SIZE])
    
#     # Debug: In thông tin batch đầu tiên
#     if i == 0:
#         print("Ví dụ một batch mẫu:")
#         for idx, code in zip(batch_indices, batch_codes):
#             print(f"Index {idx}: {code[:100]}... (độ dài: {len(code)})")
    
#     # Encode batch
#     inputs = tokenizer(batch_codes, return_tensors="pt", truncation=True, padding=True, max_length=512)
#     inputs = {k: v.to("cuda:0", non_blocking=True) for k, v in inputs.items()}

#     with torch.inference_mode(), torch.cuda.amp.autocast():
#         outputs = model(**inputs)

#     batch_vectors = outputs.last_hidden_state[:, 0, :].cpu().numpy()

#     # Lưu vào dict
#     index_to_vector_map.update({idx: vec.tolist() for idx, vec in zip(batch_indices, batch_vectors)})

#     batch_count += 1
#     if batch_count % SAVE_EVERY == 0:  # Lưu mỗi SAVE_EVERY batch
#         with open(SAVE_PATH, "wb") as f:
#             pickle.dump(index_to_vector_map, f)
#         torch.cuda.empty_cache()  
#         gc.collect()

# # Lưu lần cuối
# with open(SAVE_PATH, "wb") as f:
#     pickle.dump(index_to_vector_map, f)

# print(f"✅ Hoàn tất encoding! Đã lưu vào {SAVE_PATH}")


Ví dụ mẫu ban đầu:
Index 0 - is_vul: True - func: static int rndis_query_response(USBNetState *s,
  ... - func_before: static int rndis_query_response(USBNetState *s,
  ...
Index 1 - is_vul: True - func: static int rndis_set_response(USBNetState *s,
    ... - func_before: static int rndis_set_response(USBNetState *s,
    ...
Index 2 - is_vul: True - func: static void usb_net_handle_dataout(USBNetState *s,... - func_before: static void usb_net_handle_dataout(USBNetState *s,...
Index 3 - is_vul: True - func: static int ne2000_buffer_full(NE2000State *s)
{
  ... - func_before: static int ne2000_buffer_full(NE2000State *s)
{
  ...
Index 4 - is_vul: True - func: static int mov_read_dref(MOVContext *c, AVIOContex... - func_before: static int mov_read_dref(MOVContext *c, AVIOContex...
Tổng số mẫu sau khi lọc: 17975
Số mẫu cần encode sau khi loại bỏ checkpoint: 17975


Encoding...:   0%|          | 0/141 [00:00<?, ?it/s]

Ví dụ một batch mẫu:
Index 0: static int rndis_query_response(USBNetState *s,
                rndis_query_msg_type *buf, unsigned ... (độ dài: 1590)
Index 1: static int rndis_set_response(USBNetState *s,
                rndis_set_msg_type *buf, unsigned int ... (độ dài: 983)
Index 2: static void usb_net_handle_dataout(USBNetState *s, USBPacket *p)
{
    int sz = sizeof(s->out_buf) -... (độ dài: 1205)
Index 3: static int ne2000_buffer_full(NE2000State *s)
{
    int avail, index, boundary;

    index = s->curp... (độ dài: 345)
Index 4: static int mov_read_dref(MOVContext *c, AVIOContext *pb, MOVAtom atom)
{
    AVStream *st;
    MOVSt... (độ dài: 3855)
Index 5: void close_connection(h2o_http2_conn_t *conn)
{
    conn->state = H2O_HTTP2_CONN_STATE_IS_CLOSING;

... (độ dài: 348)
Index 6: static void parse_input(h2o_http2_conn_t *conn)
{
    size_t http2_max_concurrent_requests_per_conne... (độ dài: 1605)
Index 7: static void on_read(h2o_socket_t *sock, int status)
{
    h2o_http2_conn_t *c

Encoding...: 100%|██████████| 141/141 [02:43<00:00,  1.16s/it]

✅ Hoàn tất encoding! Đã lưu vào encoded_vectors.pkl


In [10]:
print("Tổng số mẫu:", len(mega_vul_df))
print("Số mẫu vulnerable:", len(mega_vul_df[mega_vul_df['is_vul'] == True]))
print("Số mẫu clean:", len(mega_vul_df[mega_vul_df['is_vul'] == False]))


Tổng số mẫu: 353873
Số mẫu vulnerable: 17975
Số mẫu clean: 335898


# Load encodede vectors pickle file

In [1]:
import json
import pandas as pd

# Đường dẫn file JSON gốc (cập nhật lại đường dẫn theo hệ thống của bạn)
input_file = r"C:\Users\Administrator\Downloads\mapr\VulScribeR-main\dataset\simple\megavul_vul.json"

# Load file JSON và chuyển thành DataFrame
with open(input_file, "r", encoding="utf-8") as f:
    json_data = json.load(f)
vul_df = pd.DataFrame(json_data)


In [2]:
import pickle
import pandas as pd

file_path = r"C:\Users\Administrator\Downloads\mapr\VulScribeR-main\code\encoded_vectors.pkl"

try:
    with open(file_path, "rb") as f:
        data = pickle.load(f)
    print("✅ File pickle tải thành công!", len(data))
except Exception as e:
    print("❌ Lỗi khi tải file pickle:", e)


✅ File pickle tải thành công! 17975


In [ ]:

# Chuẩn bị danh sách các mẫu chưa encode trước vòng lặp
# code_samples = [(i, str(row["func"])) for i, row in vul_df.iterrows() if str(row["func"]).strip()]
# code_samples = [(i, code) for i, code in code_samples if i not in data]  
# total_samples = len(code_samples)

KeyError: 'func'

In [3]:
# print(type(data))
# print(len(data))
encoded_code_vectors = list(data.values())
print(f"✅ Đã lấy được {len(encoded_code_vectors)} vectors từ pickle!")


✅ Đã lấy được 17975 vectors từ pickle!


In [4]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(encoded_code_vectors)

# Get the cluster labels for each code piece
cluster_labels = kmeans.labels_

c:\ProgramData\miniconda3\envs\vulscriber-cluster-env\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [7]:
import os

# Ensure the directory exists
output_dir = './dataset/container_data'
os.makedirs(output_dir, exist_ok=True)

# Gán cluster mặc định là -1
vul_df['cluster'] = -1

# Chỉ cập nhật cluster cho các index có vector
valid_indices = vul_df.iloc[:len(cluster_labels)].index
vul_df.loc[valid_indices, 'cluster'] = cluster_labels
# vul_df['cluster'] = cluster_labels

for cluster_number, group_df in vul_df.groupby('cluster'):
    file_name = f'./dataset/container_data/megavul_vuls_cls_{cluster_number}.jsonl'
    group_df[['func', 'is_vul', 'diff_line_info', 'func_before', 'file_path']].to_json(file_name, orient='records', lines=True)


In [ ]:
with open(r"C:\Users\Administrator\Downloads\mapr\VulScribeR-main\code\dataset\wrong\megavul_vuls_cls_0.jsonl", "r", encoding="utf-8-sig") as f:
    data = f.read()

with open(r"C:\Users\Administrator\Downloads\mapr\VulScribeR-main\code\dataset\wrong\test.jsonl", "w", encoding="utf-8") as f:
    f.write(data)


In [8]:
arr = [0,0,0,0,0,0]
for i in cluster_labels:
    arr[i]+=1

print(arr) # avg -> BIGVUL [1238, 1187, 1069, 1372, 3917, 0] order is [4,3,1,0,2] and kmeans indices are 0,1,2,3,4 for 5 clusters
# MEGAVUL full  [41918, 65107, 68454, 85557, 92837, 0]
# MEGAVUL VUL [5631, 3288, 3460, 1311, 4285, 0]


[5631, 3288, 3460, 1311, 4285, 0]
